<a href="http://laf-fabric.readthedocs.org/en/latest/" target="_blank"><img align="left" src="images/laf-fabric-xsmall.png"/></a>
<a href="http://emdros.org" target="_blank"><img align="left" src="files/images/Emdros-xsmall.png"/></a>
<a href="http://www.persistent-identifier.nl/?identifier=urn%3Anbn%3Anl%3Aui%3A13-048i-71" target="_blank"><img align="left"src="images/etcbc4easy-small.png"/></a>
<a href="http://www.godgeleerdheid.vu.nl/etcbc" target="_blank"><img align="right" src="images/VU-ETCBC-xsmall.png"/></a>

# Inclusion

In [1]:
import sys
import collections
import subprocess

from lxml import etree

import laf
from laf.fabric import LafFabric
from etcbc.preprocess import prepare
fabric = LafFabric()

  0.00s This is LAF-Fabric 4.4.7
API reference: http://laf-fabric.readthedocs.org/en/latest/texts/API-reference.html
Feature doc: http://shebanq-doc.readthedocs.org/en/latest/texts/welcome.html



In [17]:
API = fabric.load('etcbc4b', '--', 'ntn', {
    "xmlids": {"node": False, "edge": False},
    "features": ('''
        otype monads minmonad maxmonad
        book chapter verse number
    ''',''),
    "prepare": prepare,
    "primary": False,
}, verbose='DETAIL')
exec(fabric.localnames.format(var='fabric'))

  0.00s LOADING API: please wait ... 
  0.00s DETAIL: COMPILING m: UP TO DATE
  0.00s INFO: USING DATA COMPILED AT: 2015-05-04T13-46-20
  0.00s DETAIL: COMPILING a: UP TO DATE
  0.01s DETAIL: keep main: G.node_anchor_min
  0.01s DETAIL: keep main: G.node_anchor_max
  0.01s DETAIL: keep main: G.node_sort
  0.01s DETAIL: keep main: G.node_sort_inv
  0.01s DETAIL: keep main: G.edges_from
  0.01s DETAIL: keep main: G.edges_to
  0.02s DETAIL: keep main: F.etcbc4_db_maxmonad [node] 
  0.02s DETAIL: keep main: F.etcbc4_db_minmonad [node] 
  0.02s DETAIL: keep main: F.etcbc4_db_monads [node] 
  0.02s DETAIL: keep main: F.etcbc4_db_otype [node] 
  0.02s DETAIL: load main: F.etcbc4_ft_number [node] 
  0.94s DETAIL: load main: F.etcbc4_sft_book [node] 
  0.96s DETAIL: load main: F.etcbc4_sft_chapter [node] 
  0.98s DETAIL: load main: F.etcbc4_sft_verse [node] 
  0.99s DETAIL: prep prep: G.node_sort
  1.07s DETAIL: prep prep: G.node_sort_inv
  2.51s DETAIL: prep prep: L.node_up
    11s DETAIL: pre

In [3]:
print('"{}"'.format(F.monads.v(431935)))

"25859-25866"


In [4]:
otypes = (
    'book',
    'chapter',
    'verse',
    'half_verse',
    'sentence',
    'sentence_atom',
    'clause',
    'clause_atom',
    'phrase',
    'phrase_atom',
    'subphrase',
    'word',
)

In [5]:
Lu = {}
Ld = {}

def getmonads(attr):
    ranges = attr.split(',')
    covered = set()
    for r in ranges:
        if '-' in r:
            (start, end) = r.split('-', 1)
            for i in range(int(start), int(end)+1): covered.add(i)
        else:
            covered.add(int(r))
    return covered

def fill(er, ed):
    cur_min = None
    cur_max = None
    cur_ers = collections.deque()
    skip_ed = None
    for n in NN():
        otype = F.otype.v(n)
        if otype == er:
            this_er = n
            this_min = int(F.minmonad.v(n))
            this_max = int(F.maxmonad.v(n))
            this_monads = getmonads(F.monads.v(n))
            while len(cur_ers):
                if cur_ers[0][2] < this_min:
                    cur_ers.popleft()
                else:
                    break
            cur_ers.append((this_er, this_min, this_max, this_monads))
            cur_max = max(x[2] for x in cur_ers)
            cur_min = min(x[1] for x in cur_ers)
            skip_ed = False
        elif otype in ed:
            if len(cur_ers) > 0:
                if not skip_ed:
                    this_min = int(F.minmonad.v(n))
                    this_max = int(F.maxmonad.v(n))
                if this_min > cur_max:
                    skip_ed = True
                if this_max <= cur_max:
                    this_monads = getmonads(F.monads.v(n))
                    for (that_er, that_min, that_max, that_monads) in cur_ers:
                        if  this_monads <= that_monads:
                            Lu.setdefault(er, {})[n] = that_er
                            Ld.setdefault(otype, {}).setdefault(that_er, []).append(n)

In [6]:
lower_types = set(otypes)
for t in otypes:
    print("objects embedded in {}".format(t))
    lower_types.remove(t)
    fill(t, lower_types)

objects embedded in book
objects embedded in chapter
objects embedded in verse
objects embedded in half_verse
objects embedded in sentence
objects embedded in sentence_atom
objects embedded in clause
objects embedded in clause_atom
objects embedded in phrase
objects embedded in phrase_atom
objects embedded in subphrase
objects embedded in word


In [7]:
Ld['phrase'][431935]

[620998, 620999, 621000]

In [8]:
Ld['word'][620998]

[25858, 25859, 25861, 25862, 25863]

In [9]:
Ld['word'][620999]

[25860]

In [10]:
Ld['word'][621000]

[25864, 25865]

In [18]:
p = 612164
s = Lu['sentence'][p]
print('{} {}:{}#{} phr {}'.format(
    F.book.v(Lu['book'][s]), 
    F.chapter.v(Lu['chapter'][s]), 
    F.verse.v(Lu['verse'][s]), 
    F.number.v(s),
    F.number.v(p),
))

Genesis 25:6#9 phr 3


In [19]:
Ld['phrase'][428987]

[612160, 612161, 612164, 612165, 612166]